# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd2c8055dc0>
├── 'a' --> tensor([[ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327]])
└── 'x' --> <FastTreeValue 0x7fd2c8055d90>
    └── 'c' --> tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                        [ 0.3871, -0.9379, -0.3367, -0.6578],
                        [-0.7158, -0.5375, -0.3006, -1.8463]])

In [4]:
t.a

tensor([[ 0.0428, -1.8604, -0.5831],
        [ 1.8451, -0.4697, -1.0327]])

In [5]:
%timeit t.a

63 ns ± 0.0844 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd2c8055dc0>
├── 'a' --> tensor([[ 0.8975,  0.5825,  0.4089],
│                   [-1.0066, -0.6037, -0.9958]])
└── 'x' --> <FastTreeValue 0x7fd2c8055d90>
    └── 'c' --> tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                        [ 0.3871, -0.9379, -0.3367, -0.6578],
                        [-0.7158, -0.5375, -0.3006, -1.8463]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.0171 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327]]),
    x: Batch(
           c: tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                      [ 0.3871, -0.9379, -0.3367, -0.6578],
                      [-0.7158, -0.5375, -0.3006, -1.8463]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0428, -1.8604, -0.5831],
        [ 1.8451, -0.4697, -1.0327]])

In [11]:
%timeit b.a

53.8 ns ± 0.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.3925, -1.5232,  0.6475],
               [-0.0629,  0.0556, -0.1380]]),
    x: Batch(
           c: tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                      [ 0.3871, -0.9379, -0.3367, -0.6578],
                      [-0.7158, -0.5375, -0.3006, -1.8463]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.183 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

860 ns ± 9.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 21.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 735 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd2cd0f8190>
├── 'a' --> tensor([[[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]],
│           
│                   [[ 0.0428, -1.8604, -0.5831],
│                    [ 1.8451, -0.4697, -1.0327]]])
└── 'x' --> <FastTreeValue 0x7fd2b837e580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 98 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd1f2972fd0>
├── 'a' --> tensor([[ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327],
│                   [ 0.0428, -1.8604, -0.5831],
│                   [ 1.8451, -0.4697, -1.0327]])
└── 'x' --> <FastTreeValue 0x7fd2cd0d7070>
    └── 'c' --> tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                        [ 0.3871, -0.9379, -0.3367, -0.6578],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 38.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.1 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]],
       
               [[ 0.0428, -1.8604, -0.5831],
                [ 1.8451, -0.4697, -1.0327]]]),
    x: Batch(
           c: tensor([[[ 1.0207,  0.6295,  0.4877,  0.7561],
                       [ 0.3871, -0.9379, -0.3367, -0.6578],
                       [-0.7158, -0.5375, -0.3006, -1.8463]],
         

In [26]:
%timeit Batch.stack(batches)

79.6 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327],
               [ 0.0428, -1.8604, -0.5831],
               [ 1.8451, -0.4697, -1.0327]]),
    x: Batch(
           c: tensor([[ 1.0207,  0.6295,  0.4877,  0.7561],
                      [ 0.3871, -0.9379, -0.3367, -0.6578],
                      [-0.7158, -0.5375, -0.3006, -1.8463],
                      [ 1.0207,  0.6295,  0.4877,  0.7561],
                      [ 0.3871, -

In [28]:
%timeit Batch.cat(batches)

162 µs ± 539 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

333 µs ± 4.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
